In [0]:
!pip install selenium
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
from selenium import webdriver
import pandas as pd
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [0]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get('https://www.instagram.com/natgeo/?hl=en')

import time
from bs4 import BeautifulSoup
import re


links = []
scripts = []

while len(links) < 500:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(2)
    
    page_html = driver.page_source
    data = BeautifulSoup(page_html, 'html.parser')
    
    body = data.find('body')
    script = body.find('span')
    
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            links.append(link.get('href').lstrip('/p/').rstrip('/'))
            
    links = list(dict.fromkeys(links))

In [0]:
!pip install instaloader

In [0]:
import instaloader
L = instaloader.Instaloader()

posts = dict()

for count, post in enumerate(links):
    Post = instaloader.Post.from_shortcode(L.context, post)
    
    if not Post.is_video:
        posts[count] = [Post._asdict()['display_url'], Post.caption, Post.comments, Post.likes]

In [0]:
insta=pd.DataFrame.from_dict(posts, orient = 'index', columns = ['display_url', 'caption', 'comments', 'likes'])

In [0]:
insta.to_csv('insta.csv')

In [0]:
import pandas as pd
insta=pd.read_csv('insta.csv')

In [0]:
insta.head()

,Unnamed: 0,display_url,caption,comments,likes
0,0,https://scontent-hkg3-1.cdninstagram.com/vp/56...,Photo by @dina_litovsky | These are portraits ...,365,57910
1,1,https://scontent-hkg3-1.cdninstagram.com/vp/c3...,Photo by David Guttenfelder @dguttenfelder // ...,190,69265
2,2,https://scontent-hkg3-1.cdninstagram.com/vp/62...,"Photo by @stevewinterphoto | Our new story, “T...",1482,421468
3,3,https://scontent-hkg3-1.cdninstagram.com/vp/cc...,"Photo by @davidalanharvey | Outer Banks, North...",341,160309
4,4,https://scontent-hkg3-1.cdninstagram.com/vp/d0...,Photo by Jimmy Chin @jimmychin | Alpine dreams...,551,314242


In [0]:
!pip install google-cloud-vision

     |████████████████████████████████| 440kB 9.6MB/s 


In [0]:
#Mounting google drive to access the csv files created
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls

drive/  insta.csv  sample_data/


In [0]:
from google.cloud import vision
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My First Project-a7873dcc698e.json"
client = vision.ImageAnnotatorClient()

In [0]:
def get_labels(url):
    response = client.annotate_image({
      'image': {'source': {'image_uri': url}},
      'features': [{'type': "LABEL_DETECTION"}],
    })
    
    labels = [annotation.description for annotation in response.label_annotations]
    
    return labels

In [0]:
insta['labels'] = insta['display_url'].map(get_labels)

In [0]:
insta.head()

,Unnamed: 0,display_url,caption,comments,likes,labels
0,0,https://scontent-hkg3-1.cdninstagram.com/vp/56...,Photo by @dina_litovsky | These are portraits ...,365,57910,"[Lady, Beauty, Lip, Fashion, Photo shoot, Arm,..."
1,1,https://scontent-hkg3-1.cdninstagram.com/vp/c3...,Photo by David Guttenfelder @dguttenfelder // ...,190,69265,"[Visual effect lighting, Technology, Display d..."
2,2,https://scontent-hkg3-1.cdninstagram.com/vp/62...,"Photo by @stevewinterphoto | Our new story, “T...",1482,421468,"[Tiger, Wildlife, Vertebrate, Bengal tiger, Si..."
3,3,https://scontent-hkg3-1.cdninstagram.com/vp/cc...,"Photo by @davidalanharvey | Outer Banks, North...",341,160309,"[Wave, Ocean, Surfing Equipment, Surfing, Sea,..."
4,4,https://scontent-hkg3-1.cdninstagram.com/vp/d0...,Photo by Jimmy Chin @jimmychin | Alpine dreams...,551,314242,"[Water, Nature, Black-and-white, Atmospheric p..."


In [0]:
import pickle
pickle.dump( insta, open( "save.p", "wb" ) )

In [0]:
insta.to_csv('insta.csv')